# Traitement intermédiaire

Ce notebook permet de récupérer, depuis un fichier de sous-titre, deux nouveaux fichiers qui serviront à avoir des sous-titres segmentés en phrases :
1. Un fichier de sous-titre .vtt sur segmenté, où un sous-titre correspond exclusivement à une phrase ou à un morceau de phrases 
2. Un fichier .txt contenant une phrase par ligne

In [8]:
import spacy
import os
import re
import module_traitement as m
spacy.prefer_gpu()
nlp = spacy.load("fr_dep_news_trf")
from spacy.language import Language

In [98]:
# Pour Mediapi
file_with_path = m.lister_fichiers_with_path("../data/aligned_mediapi/")
folder = m.lister_fichiers("../data/aligned_mediapi/")
output_seg = "../data/new_segmentation_mediapi/"
output_sent = "../data/sentence_mediapi/"

In [9]:
# Pour Matignon - LSF 
file_with_path = m.lister_fichiers_with_path("../data/cr_audio_aligned/")
folder = m.lister_fichiers("../data/cr_audio_aligned/")
output_seg = "../data/new_segmentation_cr/"
output_sent = "../data/sentence_matignon/"

## Fichier sur-segmenté

Dans ces fichiers, on fait les traitements suivants : 
1. Récupérer les ponctuations fortes (point, point d'exclamation, point d'interrogation) au milieu des sous-titre
2. Garder les timestamps en mémoire
3. Avoir la durée de prononciation d'une lettre, pour découper le sous-titre en fonction
4. Couper le sous-titre et générer un nouveau fichier

### Traitement 

Le prochain traitement peut s'effectuer deux fois, au cas où il y avait plusieurs ponctations fortes. Je peux modifier l'expression régulière pour ne pas prendre les LETTRE_MAJ. dans ma segmentation.

In [12]:
import module_traitement as m

In [10]:
file_with_path = file_with_path[:2]
folder = folder [:2]
output_seg = "test/"

In [14]:
for file,name in zip(file_with_path,folder):
    print(f"TRAITEMENT {file} ---- {name}")
    dict_sub = m.get_dict_vtt(file)
    new_dict = {}
    m = 0
    #pattern = re.compile(r'([^!?.]+[.!?])(?<!\.{3})(?=\s|$)(.+)')
    #pattern = re.compile(r'([^!?.[A-Z]]+[.!?])(?<!\.{3})(?=\s|$)(.+)')
    pattern = re.compile(r'(.+[^!?.ABCDEFGHIJKLMNOPQRSTUVWXYZ]+[.!?"])(?:\s|$)(.+)')
    sous_unite = []
    for k, v in dict_sub.items():
        for kk, vv in v.items():
            if kk == "text":
                match =  re.match(pattern,vv)
                if match:
                    sous_unite = []
                    avant_ponct_txt = match.group(1).strip()
                    apres_ponct_txt = match.group(2).strip()
                    sous_unite.append(avant_ponct_txt)
                    sous_unite.append(apres_ponct_txt)
                    print(f"avant ponctuation : {avant_ponct_txt}")
                    # print(f"après ponctuation : {apres_ponct_txt}")
                    start_time_str = v["start"]
                    end_time_str = v["end"]
                    # print(f"afficher les start_time :{start_time}, et les end_time : {end_time}")
                    start_time = m.conv_str_o_time(start_time_str)
                    end_time = m.convertir_chaine_en_temps(end_time_str)
                    nb_of_carach = len(v["text"])
                    duration = end_time - start_time
                    duration_sec = duration.total_seconds()
                    # print(f"Afficher la durée du sous-titre : {duration}, en seconde : {duration_sec}")
                    sec_par_letter = duration_sec / nb_of_carach
                    # print(f"Durée d'une lettre en seconde : {sec_par_letter}")
                    duration_avant_ponct = len(match.group(1)) * sec_par_letter
                    # print(f"Durée de la sous unité de sous-titre : {duration_avant_ponct}")
                    new_transition_time = m.ajouter_secondes(start_time_str,duration_avant_ponct)
                    # print(f"Temps de transition (début du sous-titre + durée de la première séquence avant ponctuation) : {new_transition_time}")
                    if len(sous_unite) == 2:
                        if m not in new_dict:
                            new_dict[m]={"start":start_time_str,"end":new_transition_time,"text":sous_unite[0]}
                            m = m+1
                            new_dict[m]={"start":new_transition_time,"end":end_time_str,"text":sous_unite[1]}
                            m = m+1
                    if "?" in sous_unite[1][:-1] or "!" in sous_unite[1][:-1] or "." in sous_unite[1][:-1]:
                        print(vv)
                else:
                    if m not in new_dict:
                        new_dict[m]=v
                        m = m +1
                
    m.create_vtt_file(new_dict,f"{output_seg}/{name}")
    

TRAITEMENT ../data/cr_audio_aligned/-LhfYZ1ihpI.vtt ---- -LhfYZ1ihpI.vtt


AttributeError: 'int' object has no attribute 'get_dict_vtt'

## Sortir un fichier de phrases

L'idée est de sortie un fichier contenant une phrase par ligne pour pouvoir créer plus simplement le nouveau fichier de sous-titre segmenté en phrases. 

### Traitement

Utilisation de SpaCy pour récupérer les phrases

In [105]:
for files, name in zip(file_with_path,folder):
    print("Traitement",files,name)
    dict_sub = m.get_dict_vtt(files)
    text = ""
    sentences = []
    for k,v in dict_sub.items():
        for kk,vv in v.items():
            if kk=="text":
                text = text + vv + " "
    doc = nlp(text)
    assert doc.has_annotation("SENT_START")
    for sent in doc.sents:
        sentences.append(sent.text)
    with open(f"{output_sent}/{name}","w",encoding="utf-8") as output:
        for sent in sentences:
            output.write(sent+"\n")

Traitement data/aligned_mediapi/b7f2d8f0c3.vtt b7f2d8f0c3.vtt
Traitement data/aligned_mediapi/3d0b82b459.vtt 3d0b82b459.vtt
Traitement data/aligned_mediapi/44f554f914.vtt 44f554f914.vtt
Traitement data/aligned_mediapi/b9a51f4361.vtt b9a51f4361.vtt
Traitement data/aligned_mediapi/4e073949b1.vtt 4e073949b1.vtt
Traitement data/aligned_mediapi/0b1437bc85.vtt 0b1437bc85.vtt
Traitement data/aligned_mediapi/752500b761.vtt 752500b761.vtt
Traitement data/aligned_mediapi/cba6cefad2.vtt cba6cefad2.vtt
Traitement data/aligned_mediapi/41c606553f.vtt 41c606553f.vtt
Traitement data/aligned_mediapi/3f1b2118ca.vtt 3f1b2118ca.vtt
Traitement data/aligned_mediapi/ed969c3e70.vtt ed969c3e70.vtt
Traitement data/aligned_mediapi/26f1ff8385.vtt 26f1ff8385.vtt
Traitement data/aligned_mediapi/2b4b33189c.vtt 2b4b33189c.vtt
Traitement data/aligned_mediapi/ada7d10a19.vtt ada7d10a19.vtt
Traitement data/aligned_mediapi/fd41a24117.vtt fd41a24117.vtt
Traitement data/aligned_mediapi/15abfc95ae.vtt 15abfc95ae.vtt
Traiteme

### Nettoyer fichier phrase
- remonter la ponctuation forte si elle est isolée sur une ligne
- remonter les sections commençant par une virgule si elles sont isolées du reste de la phrase sur la ligne suivante

In [ ]:
files= m.lister_fichiers_with_path(output_sent)

In [110]:
ponctuations = {"!", ".", "?", "....", "...",'"',":"}

for file in files:
    with open(file, "r") as f:
        print(f"Traitement de {file}")
        ligne_precedente = ""
        liste_sent = f.readlines()
        if len(liste_sent) > 3:
            print(f"File long enough, traitement : {file} --- {len(liste_sent)}")
            i = 0
            txt = ""
            while i < len(liste_sent):
                if i < len(liste_sent) - 1:
                    if liste_sent[i + 1].strip() in ponctuations or liste_sent[i+1].startswith(","):
                        txt = txt + liste_sent[i].strip() + liste_sent[i + 1]
                        i = i + 2
                        var = True
                    else:
                        txt = txt + liste_sent[i]
                        i = i + 1
                        var = False
                else:
                    if var == False:
                        txt = txt + liste_sent[i]
                        i = i + 1
                    else:
                        i = i +1

            with open(file, "w") as f:
                print(f"Ecriture du nouveau fichier {file}")
                f.write(txt)

            print(f"{file} done")
            i = 0
        else:
            print(f"not long enough : {file}")


Traitement de data/sentence_mediapi/b7f2d8f0c3.vtt
not long enough : data/sentence_mediapi/b7f2d8f0c3.vtt
Traitement de data/sentence_mediapi/3d0b82b459.vtt
File long enough, traitement : data/sentence_mediapi/3d0b82b459.vtt --- 60
Ecriture du nouveau fichier data/sentence_mediapi/3d0b82b459.vtt
data/sentence_mediapi/3d0b82b459.vtt done
Traitement de data/sentence_mediapi/44f554f914.vtt
not long enough : data/sentence_mediapi/44f554f914.vtt
Traitement de data/sentence_mediapi/b9a51f4361.vtt
File long enough, traitement : data/sentence_mediapi/b9a51f4361.vtt --- 8
Ecriture du nouveau fichier data/sentence_mediapi/b9a51f4361.vtt
data/sentence_mediapi/b9a51f4361.vtt done
Traitement de data/sentence_mediapi/4e073949b1.vtt
not long enough : data/sentence_mediapi/4e073949b1.vtt
Traitement de data/sentence_mediapi/0b1437bc85.vtt
not long enough : data/sentence_mediapi/0b1437bc85.vtt
Traitement de data/sentence_mediapi/752500b761.vtt
File long enough, traitement : data/sentence_mediapi/752500b

- remonter ce qu'il y a après ":" si c'est isolée de la phrase sur la ligne suivante

In [111]:
ponctuations = {":"}

# Ouvrez le fichier en mode lecture
for file in files:
    with open(file,"r") as f:
        print(f"Traitement de {file}")
        ligne_precedente = ""
        liste_sent = f.readlines()
        if len(liste_sent) > 3:
            print(f"File long enough, traitement : {file} --- {len(liste_sent)}")
            i=0
            txt=""
            while i < len(liste_sent):
                if i < len(liste_sent)-1:
                    if liste_sent[i].strip().endswith(":"):
                        txt = txt + liste_sent[i].strip() + " " + liste_sent[i+1]
                        i = i+2
                        var = True
                    else:
                        txt = txt + liste_sent[i]
                        i = i +1
                        var = False
                else:
                    if var == False:
                        txt = txt + liste_sent[i]
                        i = i +1
                    else:
                        i = i +1
            with open(file,"w") as f:
                print(f"Ecriture du nouveau fichier {file}")
                f.write(txt)
            print(f"{file} done")
            i = 0
        else:
            print(f"not long enough : {file}")



Traitement de data/sentence_mediapi/b7f2d8f0c3.vtt
not long enough : data/sentence_mediapi/b7f2d8f0c3.vtt
Traitement de data/sentence_mediapi/3d0b82b459.vtt
File long enough, traitement : data/sentence_mediapi/3d0b82b459.vtt --- 60
Ecriture du nouveau fichier data/sentence_mediapi/3d0b82b459.vtt
data/sentence_mediapi/3d0b82b459.vtt done
Traitement de data/sentence_mediapi/44f554f914.vtt
not long enough : data/sentence_mediapi/44f554f914.vtt
Traitement de data/sentence_mediapi/b9a51f4361.vtt
File long enough, traitement : data/sentence_mediapi/b9a51f4361.vtt --- 8
Ecriture du nouveau fichier data/sentence_mediapi/b9a51f4361.vtt
data/sentence_mediapi/b9a51f4361.vtt done
Traitement de data/sentence_mediapi/4e073949b1.vtt
not long enough : data/sentence_mediapi/4e073949b1.vtt
Traitement de data/sentence_mediapi/0b1437bc85.vtt
not long enough : data/sentence_mediapi/0b1437bc85.vtt
Traitement de data/sentence_mediapi/752500b761.vtt
File long enough, traitement : data/sentence_mediapi/752500b